#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
#load the data

data = yfinance.download (tickers = "TATAPOWER.NS",start="2000-03-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)

data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2023-06-20,223.100006,226.350006,222.600006,224.949997,11221842
2023-06-21,225.250000,226.000000,222.600006,224.300003,6987933
2023-06-22,224.850006,225.149994,220.500000,221.100006,6857007
2023-06-23,222.000000,222.100006,215.699997,218.300003,10890426
2023-06-26,218.300003,219.350006,216.300003,218.600006,5131270


#Data preparation

In [9]:
#renaming variables
data = data.rename(columns = {'Close': 'y',
                          'Date': 'ds'})
data.tail(1)

,Open,High,Low,y,Volume
Date,,,,,
2023-06-26,218.300003,219.350006,216.300003,218.600006,5131270


#Holidays

In [6]:
#Easter
#easter_dates = df[df.Easter == 1].ds
#easter = pd.DataFrame({'holiday': 'easter',
 #                      'ds': pd.to_datetime(easter_dates),
  #                     'lower_window': -5,
   #                    'upper_window': 2})
#easter.head(2)

,holiday,ds,lower_window,upper_window
109,easter,2014-04-20,-5,2
459,easter,2015-04-05,-5,2


In [7]:
#Thanksgiving
#thanksgiving_dates = df[df.Thanksgiving == 1].ds
#thanksgiving = pd.DataFrame({'holiday': 'thanksgiving',
 #                      'ds': pd.to_datetime(thanksgiving_dates),
  #                     'lower_window': -5,
   #                    'upper_window': 2})
#thanksgiving.head(2)

,holiday,ds,lower_window,upper_window
330,thanksgiving,2014-11-27,-5,2
694,thanksgiving,2015-11-26,-5,2


In [8]:
#combining holidays
#holidays = pd.concat([easter, thanksgiving])
#holidays

,holiday,ds,lower_window,upper_window
109,easter,2014-04-20,-5,2
459,easter,2015-04-05,-5,2
816,easter,2016-03-27,-5,2
1201,easter,2017-04-16,-5,2
1551,easter,2018-04-01,-5,2
1936,easter,2019-04-21,-5,2
2293,easter,2020-04-12,-5,2
330,thanksgiving,2014-11-27,-5,2
694,thanksgiving,2015-11-26,-5,2
1058,thanksgiving,2016-11-24,-5,2


In [10]:

df['ds'] = pd.to_datetime(data.index.values,
                          format = "%Y-%m-%d")

In [23]:
len(df)

5819

#Prophet

In [14]:
m = Prophet(
            #holidays = holidays,
            seasonality_mode = 'multiplicative',
            seasonality_prior_scale = 10,
            changepoint_prior_scale = 0.05,
            holidays_prior_scale = 10)
#m.add_regressor('Christmas')
m.fit(df)

18:31:35 - cmdstanpy - INFO - Chain [1] start processing
18:31:48 - cmdstanpy - INFO - Chain [1] done processing


In [24]:
#CV
from prophet.diagnostics import cross_validation
df_cv = cross_validation(m,
                         horizon = '31 days',
                         initial = '2400 days',
                         parallel = "processes"
                        )

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
18:56:55 - cmdstanpy - INFO - Chain [1] start processing
18:56:55 - cmdstanpy - INFO - Chain [1] start processing
18:56:55 - cmdstanpy - INFO - Chain [1] start processing
18:56:55 - cmdstanpy - INFO - Chain [1] start processing
18:57:03 - cmdstanpy - INFO - Chain [1] done processing
18:57:04 - cmdstanpy - INFO - Chain [1] start processing
18:57:08 - cmdstanpy - INFO - Chain [1] done processing
18:57:08 - cmdstanpy - INFO - Chain [1] start processing
18:57:09 - cmdstanpy - INFO - Chain [1] done processing
18:57:09 - cmdstanpy - INFO - Chain [1] start processing
18:57:10 - cmdstanpy - INFO - Chain [1] done processing
18:57:10 - cmdstanpy - INFO - Chain [1] start processing
18:57:20 - cmdstanpy - INFO - Chain [1] done processing
18:57:20 - cmdstanpy - INFO - Ch

19:00:39 - cmdstanpy - INFO - Chain [1] start processing
19:00:39 - cmdstanpy - INFO - Chain [1] done processing
19:00:39 - cmdstanpy - INFO - Chain [1] start processing
19:00:40 - cmdstanpy - INFO - Chain [1] done processing
19:00:41 - cmdstanpy - INFO - Chain [1] start processing
19:00:48 - cmdstanpy - INFO - Chain [1] done processing
19:00:48 - cmdstanpy - INFO - Chain [1] done processing
19:00:49 - cmdstanpy - INFO - Chain [1] start processing
19:00:49 - cmdstanpy - INFO - Chain [1] start processing
19:00:51 - cmdstanpy - INFO - Chain [1] done processing
19:00:51 - cmdstanpy - INFO - Chain [1] start processing
19:00:57 - cmdstanpy - INFO - Chain [1] done processing
19:00:57 - cmdstanpy - INFO - Chain [1] done processing
19:00:57 - cmdstanpy - INFO - Chain [1] done processing
19:00:57 - cmdstanpy - INFO - Chain [1] start processing
19:00:57 - cmdstanpy - INFO - Chain [1] start processing
19:00:57 - cmdstanpy - INFO - Chain [1] done processing
19:00:57 - cmdstanpy - INFO - Chain [1] 

KeyboardInterrupt: 

In [16]:
#assess model
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_cv.yhat, df_cv.y)

NameError: name 'df_cv' is not defined

18:53:34 - cmdstanpy - INFO - Chain [1] done processing
18:53:39 - cmdstanpy - INFO - Chain [1] done processing
18:53:40 - cmdstanpy - INFO - Chain [1] done processing
18:53:43 - cmdstanpy - INFO - Chain [1] done processing


#Hyperparameter tuning

In [22]:
#PArameters
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': [10, 20],
                'changepoint_prior_scale': [0.05, 0.1],
            'holidays_prior_scale': [10, 20]}
grid = ParameterGrid(param_grid)

In [14]:
#Tuning  
mae = []
for params in grid:
  #build model
  m = Prophet(holidays = holidays,
            seasonality_mode = 'multiplicative',
            seasonality_prior_scale = params['seasonality_prior_scale'],
            changepoint_prior_scale = params['changepoint_prior_scale'],
            holidays_prior_scale = params['holidays_prior_scale'])
  m.add_regressor('Christmas')
  m.fit(df)

  #CV
  df_cv = cross_validation(m,
                         horizon = '31 days',
                         initial = '2400 days',
                         parallel = "processes")

  #Gather the results
  error = mean_absolute_error(df_cv.yhat, df_cv.y)
  mae.append(error)

16:32:46 - cmdstanpy - INFO - Chain [1] start processing
16:32:48 - cmdstanpy - INFO - Chain [1] done processing
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:58 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] start processing
16:33:00 - cmdstanpy - INFO - Chain [1] start processing
16:33:00 - cmdstanpy - INFO - Chain [1] start processing
16:33:05 - cmdstanpy - INFO - Ch

16:35:16 - cmdstanpy - INFO - Chain [1] start processing
16:35:16 - cmdstanpy - INFO - Chain [1] start processing
16:35:16 - cmdstanpy - INFO - Chain [1] start processing
16:35:17 - cmdstanpy - INFO - Chain [1] start processing
16:35:23 - cmdstanpy - INFO - Chain [1] done processing
16:35:24 - cmdstanpy - INFO - Chain [1] start processing
16:35:24 - cmdstanpy - INFO - Chain [1] done processing
16:35:25 - cmdstanpy - INFO - Chain [1] done processing
16:35:25 - cmdstanpy - INFO - Chain [1] done processing
16:35:25 - cmdstanpy - INFO - Chain [1] start processing
16:35:26 - cmdstanpy - INFO - Chain [1] start processing
16:35:30 - cmdstanpy - INFO - Chain [1] done processing
16:35:30 - cmdstanpy - INFO - Chain [1] done processing
16:35:31 - cmdstanpy - INFO - Chain [1] done processing


In [15]:
#gather best parameters
best_params = grid[np.argmin(mae)]
print(best_params)

{'seasonality_prior_scale': 20, 'holidays_prior_scale': 20, 'changepoint_prior_scale': 0.1}
